# Preprocessing FASTA and BIGWIG data

In [ ]:
# from basenji import genome
import pysam
from scipy.signal import find_peaks
import pyBigWig
import random
import numpy as np
# from basenji.dna_io import dna_1hot
import os
import pandas as pd
from keras.utils import to_categorical

In [ ]:
# global variables
num_bp_in_peak = 3000
num_bp_padding = 5000
down_sample = 1.0
margin = num_bp_in_peak/2 + num_bp_padding
dir_name = '/data/smooth_muscle_cells'
fasta_file = '/data/GRCh38.p3.genome.fa'
bigwig_file = '/data/TQ326-pooled_ATAC_USPD16083803_HHHJMBBXX_L4_1.merged.nodup.no_chrM_MT.tn5.fc.signal.bigwig'
peak_file = '/data/rep1-pr1_vs_rep1-pr2.overlap.bfilt.narrowPeak.gz'
test = 0.1
validation = 0.1
min_prominence = 0.7
val_chromosome = 'chr9'
test_chromosome = 'chr17'

In [ ]:
# os.mkdir(dir_name)
os.mkdir(dir_name+"/sequences")
os.mkdir(dir_name+"/peak_data")


In [ ]:
bw = pyBigWig.open(bigwig_file, 'r')
fasta_open = pysam.Fastafile(fasta_file)
# write_given_peak_data(peak_file, bw, fasta_open)

In [ ]:
chrm_peaks = {}
non_peaks = {}
bw = pyBigWig.open(bigwig_file, 'r')
fasta_open = pysam.Fastafile(fasta_file)
def determine_peaks():
    for chromosome_name, length in bw.chroms().items():
        if len(chromosome_name) < 6:
            # then doesn't include variant 
            start = num_bp_padding + num_bp_in_peak
            end = length - num_bp_padding - num_bp_in_peak
            values = bw.values(chromosome_name, start, end)
            peaks, _ = find_peaks(values, distance=num_bp_in_peak, prominence=(min_prominence, 1.0))
            chrm_peaks[chromosome_name] = peaks
            non_peaks[chromosome_name] = []
            for i in range(len(peaks) - 1):
                curr_peak_val = peaks[i]
                end_peak_val = peaks[i+1]
                while curr_peak_val + num_bp_in_peak < end_peak_val:
                    # found a non-peak that doesn't directly overlap with adjacent peak / non-peak
                    non_peaks[chromosome_name].append(curr_peak_val + num_bp_in_peak)
                    curr_peak_val += num_bp_in_peak
        break

In [ ]:
for chromosome_name in chrm_peaks:
    write_chr_data(chromosome_name, chrm_peaks[chromosome_name], non_peaks[chromosome_name], bw, fasta_open)

In [ ]:
def write_chr_data(chromosome_name, peaks, non_peaks, bw, fasta_open):
    num_peaks = int(len(peaks)*down_sample)
    num_non_peaks = int(len(non_peaks)*down_sample)
    min_val = min(num_peaks, num_non_peaks)
    peaks = np.random.choice(peaks, min_val, replace=False)
    non_peaks = np.random.choice(non_peaks, min_val, replace=False)
    validation_index = int(min_val*test)
    train_index = int(min_val*(test+validation))
    for i in range(0, min_val):
        if i == 0:
            num_sample = 0
            partition = "test"
        elif i == validation_index:
            num_sample = 0
            partition = "validation"
        elif i == train_index:
            num_sample = 0
            partition = "train"
        write_peak_data(peaks[i], bw, fasta_open, partition, num_sample, chromosome_name)
        num_sample += 1
        write_peak_data(non_peaks[i], bw, fasta_open, partition, num_sample, chromosome_name)
        num_sample += 1


In [ ]:
def one_hot_encode(seq):
    values = {'A':[1,0,0,0], 'C':[0,1,0,0], 'G':[0,0,1,0], 'T':[0,0,0,1], 'N':[0,0,0,0]}
    return_arr = np.empty((len(seq), 4))
    for i in range(return_arr.shape[0]):
        return_arr[i] = values[seq[i]]
    return return_arr
    

In [ ]:
def write_peak_data(center_of_peak, bw, fasta_open, partition, num_sample, chromosome_name):
    start = int(center_of_peak-margin)
    end = int(center_of_peak+margin)
    if end < bw.chroms()[chromosome_name] and start>1:
        peak_seq = fasta_open.fetch(chromosome_name, start, end)
        start = int(center_of_peak-num_bp_in_peak/2)
        end = int(center_of_peak+num_bp_in_peak/2)
        peak_data = bw.values(chromosome_name, start, end)
        encoding = one_hot_encode(peak_seq).astype(float)
        np.save(dir_name+"/sequences/"+partition+str(num_sample), encoding, allow_pickle=True) 
        np.save(dir_name+"/peak_data/"+partition+str(num_sample), peak_data, allow_pickle=True)


In [ ]:
def write_given_peak_data(peak_file, bw, fasta_open):
    peak_df = pd.read_csv(peak_file, sep='\t', names=['chr_name', 'start', 'end', 'peak_name', 'peak_len', 'dot', 'num1', 'num2','num3','num4'])
    unique_vals = peak_df[['chr_name', 'start', 'end']].drop_duplicates() 
    vals = unique_vals.to_numpy()
    permutation = np.random.permutation(vals)
#     validation_index = int(len(permutation)*test)
#     train_index = int(len(permutation)*(test+validation))
    train_index = -1
    test_index = -1
    val_index = -1
    for chr_name, start, end in permutation:
        if chr_name == val_chromosome:
            partition = "validation"
            val_index += 1
            num_sample = val_index
        elif chr_name == test_chromosome:
            partition = "test"
            test_index += 1
            num_sample = test_index
        else:
            partition = "train"
            train_index += 1
            num_sample = train_index
#         if i == 0:
#             num_sample = 0
#             partition = "test"
#         elif i == validation_index:
#             num_sample = 0
#             partition = "validation"
#         elif i == train_index:
#             num_sample = 0
#             partition = "train"
        center_of_peak = start + int((end - start)/2)
        write_peak_data(center_of_peak, bw, fasta_open, partition, num_sample, chr_name)
    

In [ ]:
peak_df = pd.read_csv(peak_file, sep='\t', names=['chr_name', 'start', 'end', 'peak_name', 'peak_len', 'dot', 'num1', 'num2','num3','num4'])


In [ ]:
diffs = peak_df['end'] - peak_df['start']
diffs = diffs[diffs < 2500]

print(diffs.min())
print(diffs.max())


In [ ]:
import matplotlib.pyplot as plt
plt.hist(list(diffs.values))
plt.xlabel("length of peak")
plt.ylabel("frequency")
plt.show()

In [ ]:
len(diffs)